# Regional Allele Frequency Calculation script for the gnomAD dataset

*Step 1:* Download gnomAD v2.1 exome data from: https://gnomad.broadinstitute.org/downloads

In [ ]:
#!command wget https://gnomad-public-us-east-1.s3.amazonaws.com/release/2.1.1/vcf/exomes/gnomad.exomes.r2.1.1.sites.vcf.bgz
#!command wget https://gnomad-public-us-east-1.s3.amazonaws.com/release/2.1.1/vcf/exomes/gnomad.exomes.r2.1.1.sites.vcf.bgz.tbi

Step 2: Using BCFtools filter the gnomAD file for indels only, select required columns and also create another file with Gene Symbols

In [ ]:

#!command bcftools view --include 'TYPE="INDEL"' gnomad.exomes.r2.1.1.sites.vcf.bgz -O b >gnomad.exomes.r2.1.1.sites_indelsonly.vcf.gz

#Saving a file with AC & AN columns
!command bcftools query -f '%CHROM\t%POS\t%REF\t%ALT\t%AC\t%AN\n' gnomad.exomes.r2.1.1.sites_indelsonly.vcf.gz >gnomad.exomes.r2.1.1.sites_indelsonly_AC_AN_noheader.txt
!command echo -e "CHR\tPOS\tREF\tALT\tAC\tAN" >header.txt
!command cat header.txt gnomad.exomes.r2.1.1.sites_indelsonly_AC_AN_noheader.txt >gnomad.exomes.r2.1.1.sites_indelsonly_AC_AN.txt
!command rm gnomad.exomes.r2.1.1.sites_indelsonly_AC_AN_noheader.txt header.txt

#Saving another file with associated gene names 
!command bcftools +split-vep gnomad.exomes.r2.1.1.sites_indelsonly.vcf.gz -f '%CHROM\t%POS\t%REF\t%ALT\t%SYMBOL\n' -a vep | cut -d, -f1 >gnomad.exomes.r2.1.1.sites_indelsonly.vcf_withFIRSTSYMBOL_noheader.txt
!command echo -e "CHR\tPOS\tREF\tALT\tSYMBOL" >header.txt
!command cat header.txt gnomad.exomes.r2.1.1.sites_indelsonly.vcf_withFIRSTSYMBOL_noheader.txt >gnomad.exomes.r2.1.1.sites_indelsonly.vcf_withFIRSTSYMBOL.txt
!command rm gnomad.exomes.r2.1.1.sites_indelsonly.vcf_withFIRSTSYMBOL_noheader.txt header.txt


Setting up paramters

In [ ]:
input_file="gnomad.exomes.r2.1.1.sites_indelsonly_AC_AN.txt"
out_prefix="gnomad.exomes.r2.1.1.sites_indelsonly_rAF"
#rare frequency cut off
AF=0.0001
#Genomic Ranges
bp=[10,20,30,40]

Step 3: Create BINs for different genomic ranges (10bp,20bp,30bp,40bp)

i) Load the file using pandas

In [ ]:
import pandas as pd
from functools import reduce
df=pd.read_csv(input_file,sep="\t",dtype={'CHR': 'str', 'POS': 'int','REF':'str','ALT':'str','AC':'int','AN':'int'})

In [ ]:
df = df[df['AN'] != 0 ]
df = df[df['AC'] != 0]
df['indel_length'] = abs(df['REF'].astype(str).map(len)-df['ALT'].astype(str).map(len))
df = df[df['indel_length']<51]
df = df.reset_index(drop= True)

ii) Functions to group indels falling within the given region into same bins and assign a unique bin ID

In [ ]:
def cluster(pos_list, gap):
    maxgap=gap/2
    pos=pos_list.astype(int)
    pos.sort_values(ascending=True, inplace=True)
    groups = [[list(pos)[0]]]
    for x in pos[1:]:
        if abs(x - groups[-1][-1]) <= maxgap:
            groups[-1].append(x)
        else:
            groups.append([x])
    return groups

def group_ids(pairs,chr,bp):
    groups = cluster(pairs['POS'], bp)
    d = {}
    for i, group in enumerate(sorted(groups)):
        for elem in group:
            d[elem] = "chr"+str(chr)+"_"+str(i)
    return d


In [ ]:
chrom=df['CHR'].unique()

chrom_output=[]
creategroups={}
binned_groups={}
for i in chrom:
    input=df[df["CHR"]==i]
    for j in bp:
        creategroups[j]=group_ids(input,i,j)
        binned_groups[j] = pd.DataFrame(creategroups[j].items(), columns=['POS',"GID_bp"+str(j)])
        binned_groups[j]['POS']=binned_groups[j]['POS'].astype(int)
        
    #define list of DataFrames
    dfs = [input,binned_groups[10], binned_groups[20], binned_groups[30], binned_groups[40]]
    
    #merge all dataframes into one
    final_df = reduce(lambda  left,right: pd.merge(left,right,on=['POS'], how='outer'), dfs)
    #create bin level AC
    for j in bp:
        final_df["AC_bp"+str(j)]= final_df['AC'].groupby(final_df["GID_bp"+str(j)]).transform('sum')
    chrom_output.append(final_df)
df_rAF = pd.concat(chrom_output)
df_rAF['POS']=df_rAF['POS'].astype(str)

Step 4: Calculate sAF = AC/AN

In [ ]:
df_rAF['sAF'] = (df_rAF['AC']/df_rAF['AN'])/2

Step 5: Calculate rAN value for different values by getting the mean of AN of the indels inside the same bin, Use the rAN to calculate rAF = rAC/rAN

In [ ]:
for j in bp:
    df_rAF["AN_bp"+str(j)] = df_rAF.groupby(['CHR',"GID_bp"+str(j)])['AN'].transform('mean')
    df_rAF["rAF_bp"+str(j)] = (df_rAF["AC_bp"+str(j)]/df_rAF["AN_bp"+str(j)])/2


Step 5: Save the output to a file

In [ ]:
#Recreate the VarID column to further use
df_rAF['VarID'] = df_rAF['CHR']+"-"+df_rAF['POS']+"-"+df_rAF['REF']+"-"+df_rAF['ALT']
# Remove columns 'CHR', 'POS', 'REF', 'ALT' for reducing file size. 
df_rAF=df_rAF.drop(['CHR', 'POS','REF','ALT'], axis=1)
df_rAF = df_rAF[ ['VarID'] + [ col for col in df_rAF.columns if col != 'VarID' ] ]

df_rAF.to_csv(out_prefix+"_lt50bp.csv",index = False)

Step 6: Subset for suspicious indels (SI) (sAF ≤ 10^-4 & rAF > 10^-4) 

In [ ]:
df_rAF_SI={}
for i in bp:
    df_rAF_SI[i] = df_rAF[ (df_rAF['sAF']<=AF) & (df_rAF["rAF_bp"+str(i)]>AF) ]
    df_rAF_SI[i].to_csv(out_prefix+"_bp"+str(i)+"_SuspiciousIndels.lt50bp.csv",index = False)


Step 7: Subset for rare indels (sAF ≤ 10^-4 & rAF ≤ 10^-4)

In [ ]:
df_rAF_rare={}
for i in bp:
    df_rAF_rare[i] = df_rAF[ (df_rAF['sAF']<=AF) & (df_rAF["rAF_bp"+str(i)]<=AF) ]
    df_rAF_rare[i].to_csv(out_prefix+"_bp"+str(i)+"_RareIndels.lt50bp.csv",index = False)


Step 8: Subset for common indels (sAF > 10^-4)

In [ ]:
df_rAF_common={}
for i in bp:
    df_rAF_common[i] = df_rAF[ (df_rAF['sAF']>AF) ]
    df_rAF_common[i].to_csv(out_prefix+"_bp"+str(i)+"_CommonIndels.lt50bp.csv",index = False)


In [ ]:
df_rAF
